In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
myCV = PdfReader("myCV/Vinod.pdf")
myCV_Text = ""
for page in myCV.pages:
    text = page.extract_text()
    if text:
        myCV_Text += text

In [4]:
# comment out after initial check
# print(myCV_Text)

In [5]:
name = "Vinod"

In [6]:
system_prompt = f"""
You represent {name}. The professional summary and details of {name} are here: {myCV_Text}. 
You will be contacted by recruiters and potential employers, wanting to know more about you and your experience. 
    Be very clear and professional.
"""


In [7]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    answer_for_question = response.choices[0].message.content
    return answer_for_question

In [8]:
# Test once now
# Comment out after testing once

# messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "what is your experience in Android development ?"}]
# response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
# answer_for_question = response.choices[0].message.content
# print(answer_for_question)

In [9]:
# Test once now using frontend
# Comment out after testing once
# gr.ChatInterface(chat, type="messages").launch()

In [10]:
# the response by the agent will be evaluated here
# this is the system prompt for the evaluation action

evaluator_system_prompt = f"""
  You are an evalutor who evaluates if the response by the agent acting as {name}, to the query by a recruiter / potential employer
  is acceptable. The answer by the agent should be only from {myCV_Text}.
"""

In [11]:
def full_details_of_chat(reply, message, history):
    full_details = f"This is the history of the chat between the agent and the recruiter: \n\n{history}\n\n"
    full_details += f"This is the latest query by the recruiter: \n\n {message} \n\n"
    full_details += f"This is the latest reply by the agent that you need to evaluate: \n\n {reply} \n\n"
    return full_details

In [12]:
from pydantic import BaseModel

class EvaluationFormat(BaseModel):
    is_acceptable: bool
    feedback: str

In [13]:
def evaluate_chat(reply, message, history) -> EvaluationFormat:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": full_details_of_chat(reply, message, history)}]
    evaluation = openai.beta.chat.completions.parse(model = "gpt-4o-mini", messages = messages, response_format = EvaluationFormat)
    extracted_evaluation = evaluation.choices[0].message.parsed
    return extracted_evaluation

In [14]:
# Test the evaluation once now
# Give a false reply and check
# Comment out after testing

# reply = "I have extensive experience in Android development."
# message = "What is your experience in Android development ?"
# history = {myCV_Text}

# messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": full_details_of_chat(reply, message, history)}]
# evaluation = openai.beta.chat.completions.parse(model = "gpt-4o-mini", messages = messages, response_format = EvaluationFormat)
# extracted_evaluation = evaluation.choices[0].message.parsed
# print(extracted_evaluation)

In [15]:
# Test the evaluation once again
# Give a correct reply now and check
# Comment out after testing

# reply = "I have no experience in Android development. My specialty is Data Analytics. Do you want me to elaborate on any of my Data Analaytics projects ?"
# message = "What is your experience in Android development ?"
# history = {myCV_Text}

# messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": full_details_of_chat(reply, message, history)}]
# evaluation = openai.beta.chat.completions.parse(model = "gpt-4o-mini", messages = messages, response_format = EvaluationFormat)
# extracted_evaluation = evaluation.choices[0].message.parsed
# print(extracted_evaluation)

In [16]:
# Now enhance the chat function to include the evaluate function

def chat(message, history):
    
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    # to test failed evaluation
    # every reply will have "I have 40 years experience in android development." 
    # Because this is not present in CV, evaluation will fail every time
    # comment out after testing
    # answer_for_question = response.choices[0].message.content + " I have 40 years experience in android development."
    answer_for_question = response.choices[0].message.content
    
    answer_for_question_evaluation = evaluate_chat(answer_for_question, message, history)
    if (answer_for_question_evaluation.is_acceptable):
        print('Reply evaluation passed')
        return answer_for_question
    else:
        print('Reply evaluation failed')
        return "I am not happy with this answer: " + answer_for_question

In [17]:
# Test now
# comment out after testing
# gr.ChatInterface(chat, type="messages").launch()

In [18]:
# If evaluation fails, the agent has to retry and give a correct response
# below function derives system prompt for the retry action

def retry_system_prompt(reply, message, history, feedback):
    system_prompt_for_rerun = f"""
    Reply from agent was evaluated as Not Acceptable by the evaluator.
    This was the query: \n\n{message}
    This was the reply: \n\n{reply}
    This is the full history of the chat: \n\n{history}
    This is the feedback from the evaluator: \n\n{feedback}
    You need to try again and get a better / accurate / professional reply.
    """
    return system_prompt_for_rerun

In [19]:
# If evaluation fails, the agent has to retry and give a correct response

def retry(reply, message, history, feedback):
    messages = [{"role":"system", "content": retry_system_prompt(reply, message, history, feedback)}] + history + [{"role":"user","content":message}]
    updated_response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    extracted_updated_response = updated_response.choices[0].message.content
    return extracted_updated_response

In [20]:
# Now enhance the chat function to include the retry function

def chat(message, history):
    
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    # to test failed evaluation
    # every reply will have "I have 40 years experience in android development." 
    # Because this is not present in CV, evaluation will fail every time
    # comment out after testing
    # answer_for_question = response.choices[0].message.content + " I have 40 years experience in android development."
    answer_for_question = response.choices[0].message.content
    
    answer_for_question_evaluation = evaluate_chat(answer_for_question, message, history)
    if (answer_for_question_evaluation.is_acceptable):
        print('Reply evaluation passed')
        return answer_for_question
    else:
        print('Reply evaluation failed')
        print('I will retry')
        answer_after_retry = retry(answer_for_question, message, history, answer_for_question_evaluation.feedback)
        return answer_after_retry


In [21]:
# test now
# comment out after testing
# gr.ChatInterface(chat, type="messages").launch()

In [22]:
# To simulate evaluation failure and rerun
# idea is initially, teh answer will be in Pig Latin. Evaluator will fail it. Then retry will be in English

def chat(message, history):
    if "certifications" in message:
        new_system_prompt = "I have experience in satellite development"
    else:
        new_system_prompt = system_prompt
    messages = [{"role": "system", "content": new_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    answer_for_question = response.choices[0].message.content
    
    answer_for_question_evaluation = evaluate_chat(answer_for_question, message, history)
    if (answer_for_question_evaluation.is_acceptable):
        print('Reply evaluation passed')
        return answer_for_question
    else:
        print('Reply evaluation failed')
        print('The evaluation feedback is: ', answer_for_question_evaluation.feedback)
        print('I will retry')
        answer_after_retry = retry(answer_for_question, message, history, answer_for_question_evaluation.feedback)
        return answer_after_retry



In [24]:
# test now
# comment out after testing
# gr.ChatInterface(chat, type="messages").launch()

In [25]:
# Testing was successful
# Removing the error simulation now.
# This is the final chat function

def chat(message, history):
    
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model = "gpt-4o-mini", messages = messages)
    answer_for_question = response.choices[0].message.content
    
    answer_for_question_evaluation = evaluate_chat(answer_for_question, message, history)
    if (answer_for_question_evaluation.is_acceptable):
        print('Reply evaluation passed')
        return answer_for_question
    else:
        print('Reply evaluation failed')
        print('I will retry')
        answer_after_retry = retry(answer_for_question, message, history, answer_for_question_evaluation.feedback)
        return answer_after_retry


In [ ]:
# The final test
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Reply evaluation failed
I will retry
Reply evaluation passed
